In [109]:
import json
from matplotlib import pyplot as plt
from collections import defaultdict
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error
import numpy
import random
import gzip
import math
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from collections import Counter

In [110]:
f = gzip.open("cleaned_dataset.json.gz")
dataset = []
for l in f:
    dataset.append(json.loads(l))
dataset = dataset[0]

In [111]:
dataset[0]

{'fit': 'fit',
 'bust size': '34d',
 'item_id': '2260466',
 'weight': 137,
 'rating': 10,
 'rented for': 'vacation',
 'review_text': "An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.",
 'body type': 'hourglass',
 'review_summary': 'So many compliments!',
 'category': 'romper',
 'height': 68,
 'size': 14,
 'age': 28,
 'fit_numeric': 0}

In [112]:
dataTest = dataset[117105:]
dataTrain = dataset[:117105]

print(len(dataTrain))
print(len(dataTest))

117105
29276


In [113]:
# one hot encoding body type

unique_body_types = list(set(d['body type'] for d in dataTrain))
body_type_to_onehot = {bt: [1 if i == idx else 0 for i in range(len(unique_body_types))] for idx, bt in enumerate(unique_body_types)}

In [114]:
# using the frequency of each item id as input

item_id_counts = Counter(d['item_id'] for d in dataTrain)
total_items = sum(item_id_counts.values())

item_id_to_frequency = {item: count / total_items for item, count in item_id_counts.items()}


In [115]:
def feat1(datum):
    
    onehot_body_type = body_type_to_onehot.get(datum['body type'], [0]*len(unique_body_types))
    
    frequency_item_id = item_id_to_frequency.get(datum['item_id'], 0)
    
    height = datum['height']
    weight = datum['weight']
    size = datum['size']
    
    return [1] + [height] + [weight] + [size] + onehot_body_type + [frequency_item_id]

In [116]:
X_train = [feat1(d) for d in dataTrain]
y_train = [d['fit_numeric'] for d in dataTrain]

X_test = [feat1(d) for d in dataTest]
y_test = [d['fit_numeric'] for d in dataTest]


In [117]:
model = LogisticRegression(max_iter=500, solver = 'saga', verbose=1)
model.fit(X_train, y_train)

Epoch 1, change: 1.00000000
Epoch 2, change: 0.06870177
Epoch 3, change: 0.06154894
Epoch 4, change: 0.05924740
Epoch 5, change: 0.05026000
Epoch 6, change: 0.04726574
Epoch 7, change: 0.04443500
Epoch 8, change: 0.04199048
Epoch 9, change: 0.03901402
Epoch 10, change: 0.03666024
Epoch 11, change: 0.03462586
Epoch 12, change: 0.03250560
Epoch 13, change: 0.03152542
Epoch 14, change: 0.03064629
Epoch 15, change: 0.02981865
Epoch 16, change: 0.02899516
Epoch 17, change: 0.02819683
Epoch 18, change: 0.02742270
Epoch 19, change: 0.02667889
Epoch 20, change: 0.02594178
Epoch 21, change: 0.02521656
Epoch 22, change: 0.02453319
Epoch 23, change: 0.02384364
Epoch 24, change: 0.02319355
Epoch 25, change: 0.02252822
Epoch 26, change: 0.02192325
Epoch 27, change: 0.02131147
Epoch 28, change: 0.02072585
Epoch 29, change: 0.02013973
Epoch 30, change: 0.01957439
Epoch 31, change: 0.01902982
Epoch 32, change: 0.01849817
Epoch 33, change: 0.01781030
Epoch 34, change: 0.01712500
Epoch 35, change: 0.016

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(max_iter=500, solver='saga', verbose=1)

In [118]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy", train_accuracy)
print(f"Test Accuracy", test_accuracy)

Training Accuracy 0.7346569318133299
Test Accuracy 0.7357904085257548
